In [9]:
import os
import glob
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.utils import save_image, make_grid

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## DataLoader

In [2]:
class LabeledImageDataset(Dataset):
    def __init__(self, image_folder):
        self.image_folder = image_folder
        self.image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

        self.transform = transforms.Compose([
            transforms.Grayscale(),
            transforms.Resize((414, 624)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))  # Scaling pixel values
        ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_folder, img_name)

        # Load and transform image
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)

        # Extract label from filename
        label = float(os.path.splitext(img_name)[0])
        label = torch.tensor([label], dtype=torch.float32)

        return image, label


In [3]:
image_folder = "/kaggle/input/lakedata/Images" 
batch_size = 32

dataset = LabeledImageDataset(image_folder)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Sample usage:
for imgs, labels in dataloader:
    print("Batch images:", imgs.shape)
    print("Labels:", labels[:5])         
    break


Batch images: torch.Size([32, 1, 414, 624])
Labels: tensor([[21.0784],
        [28.9665],
        [24.0093],
        [29.0588],
        [16.1568]])


## Generator

In [4]:
class Generator(nn.Module):
    def __init__(self, latent_dim, label_dim=1):
        super(Generator, self).__init__()

        self.init_h, self.init_w = 25, 39  # 25×16 = 400 height, will upsample to 414
        self.init_channels = 128

        self.project = nn.Sequential(
            nn.Linear(latent_dim + label_dim, self.init_channels * self.init_h * self.init_w),
            nn.BatchNorm1d(self.init_channels * self.init_h * self.init_w),
            nn.ReLU(True)
        )

        self.upsampler = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),   # 25×39 → 50×78
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),    # 50×78 → 100×156
            nn.BatchNorm2d(32),
            nn.ReLU(True),

            nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding=1),    # 100×156 → 200×312
            nn.BatchNorm2d(16),
            nn.ReLU(True),

            nn.ConvTranspose2d(16, 1, kernel_size=4, stride=2, padding=1),     # 200×312 → 400×624
            nn.Tanh(),

            nn.Upsample(size=(414, 624), mode='bilinear', align_corners=False)  # 400→414
        )

    def forward(self, z, labels):
        x = torch.cat((z, labels), dim=1)
        x = self.project(x)
        x = x.view(x.size(0), self.init_channels, self.init_h, self.init_w)
        return self.upsampler(x)


### Checking the output dimensions of generator

In [5]:
latent_dim = 100
generator = Generator(latent_dim).to(device)

# Example
z = torch.randn(8, latent_dim).to(device)
labels = torch.rand(8, 1).to(device) * 2 - 1  # Random normalized labels in [-1, 1]
fake_images = generator(z, labels)
print(fake_images.shape)


torch.Size([8, 1, 414, 624])


## Discriminator

In [6]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.conv_blocks = nn.Sequential(
            nn.Conv2d(2, 64, kernel_size=4, stride=2, padding=1),  # (N, 2, H, W) → (N, 64, H/2, W/2)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),  # → (N, 128, H/4, W/4)
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),  # → (N, 256, H/8, W/8)
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
        )

        # Normalize feature map to fixed shape
        self.adaptive_pool = nn.AdaptiveAvgPool2d((16, 16))  # Force output shape to (256, 16, 16)

        self.flatten = nn.Flatten()
        self.classifier = nn.Sequential(
            nn.Linear(256 * 16 * 16, 1),  # matches output of adaptive pooling
            nn.Sigmoid()
        )

    def forward(self, img, labels):
        # labels: (N, 1) → reshape to (N, 1, H, W)
        label_map = labels.view(-1, 1, 1, 1).expand(-1, 1, img.size(2), img.size(3))
        x = torch.cat((img, label_map), dim=1)  # concat along channel dimension → (N, 2, H, W)

        features = self.conv_blocks(x)
        pooled = self.adaptive_pool(features)
        flat = self.flatten(pooled)
        out = self.classifier(flat)
        return out


### Checking Discrimintor 

In [7]:
D = Discriminator().to(device)

# creating dummy grayscale images
dummy_images = torch.randn(8, 1, 414, 624).to(device)

# crerating dummy variables
dummy_labels = torch.rand(8, 1).to(device) * 2 - 1  #normalising

# Run forward pass
with torch.no_grad():
    output = D(dummy_images, dummy_labels)

print("Discriminator output shape:", output.shape)
print("Discriminator output values:", output.squeeze())


Discriminator output shape: torch.Size([8, 1])
Discriminator output values: tensor([0.5036, 0.5026, 0.5209, 0.5437, 0.5385, 0.5330, 0.5437, 0.5394],
       device='cuda:0')


## Training Loop

In [8]:
# Hyperparameters
num_epochs = 20
batch_size = 32
latent_dim = 100
sample_labels = torch.tensor([2.5, 5.0, 6.5, 7.5]).view(-1, 1).to(device)  # example labels

# Initialize models
G = Generator(latent_dim).to(device)
D = Discriminator().to(device)

# Loss and optimizers
criterion = nn.BCELoss()
optimizer_G = optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Training loop
for epoch in range(num_epochs):
    for batch_idx, (real_images, real_labels) in enumerate(dataloader):
        real_images = real_images.to(device)
        real_labels = real_labels.to(device).view(-1, 1)

        batch_size = real_images.size(0)

        # Create labels
        real_targets = torch.ones(batch_size, 1).to(device)
        fake_targets = torch.zeros(batch_size, 1).to(device)

        # ======== Train Discriminator ========
        optimizer_D.zero_grad()

        # Real images
        real_preds = D(real_images, real_labels)
        d_real_loss = criterion(real_preds, real_targets)

        # Fake images
        z = torch.randn(batch_size, latent_dim).to(device)
        fake_images = G(z, real_labels)
        fake_preds = D(fake_images.detach(), real_labels)
        d_fake_loss = criterion(fake_preds, fake_targets)

        # Total discriminator loss
        d_loss = d_real_loss + d_fake_loss
        d_loss.backward()
        optimizer_D.step()

        # ======== Train Generator ========
        optimizer_G.zero_grad()

        z = torch.randn(batch_size, latent_dim).to(device)
        gen_images = G(z, real_labels)
        preds = D(gen_images, real_labels)
        g_loss = criterion(preds, real_targets)

        g_loss.backward()
        optimizer_G.step()

        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}] Batch [{batch_idx}] "
                  f"D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

    # ======== Save Sample Images Each Epoch ========
    with torch.no_grad():
        z = torch.randn(sample_labels.size(0), latent_dim).to(device)
        generated_samples = G(z, sample_labels)
        generated_samples = (generated_samples + 1) / 2  # [-1, 1] to [0, 1]
        grid = make_grid(generated_samples, nrow=len(sample_labels), normalize=True)
        os.makedirs("generated_samples", exist_ok=True)
        save_image(grid, f"generated_samples/epoch_{epoch+1}.png")
        print(f"Saved sample images for epoch {epoch+1}")


Epoch [1/20] Batch [0] D Loss: 1.3975, G Loss: 1.8265
Epoch [1/20] Batch [100] D Loss: 1.0256, G Loss: 3.0023
Epoch [1/20] Batch [200] D Loss: 0.8638, G Loss: 1.3935
Saved sample images for epoch 1
Epoch [2/20] Batch [0] D Loss: 1.2035, G Loss: 0.5170
Epoch [2/20] Batch [100] D Loss: 1.1154, G Loss: 0.5311
Epoch [2/20] Batch [200] D Loss: 1.1066, G Loss: 0.7154
Saved sample images for epoch 2
Epoch [3/20] Batch [0] D Loss: 1.2034, G Loss: 1.3006
Epoch [3/20] Batch [100] D Loss: 1.1822, G Loss: 1.0421
Epoch [3/20] Batch [200] D Loss: 1.2227, G Loss: 0.9446
Saved sample images for epoch 3
Epoch [4/20] Batch [0] D Loss: 1.0574, G Loss: 0.8220
Epoch [4/20] Batch [100] D Loss: 1.3902, G Loss: 0.5601
Epoch [4/20] Batch [200] D Loss: 1.4010, G Loss: 1.1919
Saved sample images for epoch 4
Epoch [5/20] Batch [0] D Loss: 1.5331, G Loss: 1.3680
Epoch [5/20] Batch [100] D Loss: 1.2968, G Loss: 1.1087
Epoch [5/20] Batch [200] D Loss: 1.3076, G Loss: 0.8323
Saved sample images for epoch 5
Epoch [6/2